<a href="https://colab.research.google.com/github/DorotaJanosz/machine-learning-bootcamp/blob/master/unsupervised/03_association_rules/01_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import bibliotek

In [55]:
import numpy as np
import pandas as pd

###Wygenerowanie danych

In [56]:
data = {'produkty': ['chleb jajka mleko', 'mleko ser', 'chleb masło ser', 'chleb jajka']}

transactions = pd.DataFrame(data=data, index=[1, 2, 3, 4])
transactions

,produkty
1,chleb jajka mleko
2,mleko ser
3,chleb masło ser
4,chleb jajka


###Przygotowanie danych

In [57]:
# rozwinięcie kolumny do obiektu DataFrame

expand = transactions['produkty'].str.split(expand=True) 
expand

,0,1,2
1,chleb,jajka,mleko
2,mleko,ser,None
3,chleb,masło,ser
4,chleb,jajka,None


In [58]:
# wydobycie nazw wszystkich produktów

products = []
for col in expand.columns:
  for product in expand[col].unique():
    if product is not None and product not in products:
      products.append(product)
products.sort()
products

['chleb', 'jajka', 'masło', 'mleko', 'ser']

In [59]:
transactions_encoded = np.zeros((len(transactions), len(products)), dtype='int8')
transactions_encoded

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int8)

In [60]:
for row in zip(range(len(transactions)), transactions_encoded, expand.values):
  print(row)

(0, array([0, 0, 0, 0, 0], dtype=int8), array(['chleb', 'jajka', 'mleko'], dtype=object))
(1, array([0, 0, 0, 0, 0], dtype=int8), array(['mleko', 'ser', None], dtype=object))
(2, array([0, 0, 0, 0, 0], dtype=int8), array(['chleb', 'masło', 'ser'], dtype=object))
(3, array([0, 0, 0, 0, 0], dtype=int8), array(['chleb', 'jajka', None], dtype=object))


In [61]:
#kolowanie 0-1

for row in zip(range(len(transactions)), transactions_encoded, expand.values):
  for idx, product in enumerate(products):
    if product in row[2]:
      transactions_encoded[row[0], idx] = 1
transactions_encoded

array([[1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [1, 1, 0, 0, 0]], dtype=int8)

In [62]:
transactions_encoded_df = pd.DataFrame(transactions_encoded, columns=products)
transactions_encoded_df

,chleb,jajka,masło,mleko,ser
0,1,1,0,1,0
1,0,0,0,1,1
2,1,0,1,0,1
3,1,1,0,0,0


###Algorytm Apriori

In [63]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support=0.0, use_colnames=True)  
supports

,support,itemsets
0,0.75,(chleb)
1,0.50,(jajka)
2,0.25,(masło)
3,0.50,(mleko)
4,0.50,(ser)
5,0.50,"(jajka, chleb)"
6,0.25,"(masło, chleb)"
7,0.25,"(mleko, chleb)"
8,0.25,"(ser, chleb)"
9,0.00,"(masło, jajka)"


In [64]:
supports = apriori(transactions_encoded_df, min_support=0.3, use_colnames=True)
supports

,support,itemsets
0,0.75,(chleb)
1,0.50,(jajka)
2,0.50,(mleko)
3,0.50,(ser)
4,0.50,"(jajka, chleb)"


In [65]:
rules = association_rules(supports, metric='confidence', min_threshold=0.65) 
rules = rules.iloc[:, [0, 1, 4, 5, 6]] 
rules

,antecedents,consequents,support,confidence,lift
0,(jajka),(chleb),0.5,1.000000,1.333333
1,(chleb),(jajka),0.5,0.666667,1.333333


In [66]:
rules.sort_values(by='lift', ascending=False)

,antecedents,consequents,support,confidence,lift
0,(jajka),(chleb),0.5,1.000000,1.333333
1,(chleb),(jajka),0.5,0.666667,1.333333
